In [29]:
import json

import pandas as pd
from pandas import json_normalize
import datetime

with open("./data2.json") as f:
    json_data = json.load(f)["classes"]

df = pd.DataFrame(json_data)
df = df[
    ["crse_id", "class_section", "enrollment_total", "instruction_mode", "strm", "subject", "subject_descr", "meetings"]
]

df = df.explode("meetings")
meetings = json_normalize(df["meetings"])
meetings = meetings[["days", "start_time", "end_time"]]
df = df.drop(columns=["meetings"]).reset_index(drop=True)
df = pd.concat([df, meetings], axis=1)

df = df[df.days != "TBA"]

df.start_time = pd.to_datetime(df.start_time, format="%H.%M.%S.000000").dt.time
df.end_time = pd.to_datetime(df.end_time, format="%H.%M.%S.000000").dt.time

df["mo"] = df.days.str.contains("Mo")
df["tu"] = df.days.str.contains("Tu")
df["we"] = df.days.str.contains("We")
df["th"] = df.days.str.contains("Th")
df["fr"] = df.days.str.contains("Fr")

# df.drop(columns=["days"], inplace=True)

min_time = pd.Timestamp(df.start_time.min().strftime("%H:%M:%S"))
max_time = pd.Timestamp(df.end_time.max().strftime("%H:%M:%S"))
time_range = pd.date_range(min_time, max_time, freq="1min").time
timeSlotTable = pd.DataFrame(time_range).rename(columns={0: "time"})

termTable = pd.DataFrame(df.strm.unique())
weekdayTable = pd.DataFrame(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]).rename(columns={0: "weekday"})

weekday_mapping = {"Mo": 1, "Tu": 2, "We": 3, "Th": 4, "Fr": 5}

courseTable = df[["crse_id", "subject"]].drop_duplicates().rename(columns={"crse_id": "courseId", "subject": "major"})
sessionTable = df[["crse_id", "strm", "class_section", "instruction_mode"]].reset_index(drop=True).rename(columns= {"crse_id": "courseId", "strm": "termId", "class_section": "section", "instruction_mode": "instructionMode"})

occupancyTable = pd.DataFrame()
for index, row in df.iterrows():
    start_time = pd.Timestamp(row.start_time.strftime("%H:%M:%S"))
    end_time = pd.Timestamp(row.end_time.strftime("%H:%M:%S"))  
    time_index = pd.date_range(start_time, end_time, freq="1min").time

    for weekday in ["Mo", "Tu", "We", "Th", "Fr"]:
        if weekday not in row["days"]:
            continue
        occSubTable = pd.DataFrame()
        occSubTable["time"] = time_index
        occSubTable["sessionId"] = row["strm"]
        occSubTable["courseId"] = row["crse_id"]
        occSubTable["weekdayId"] = weekday_mapping[weekday]
        occSubTable["studentCount"] = row["enrollment_total"]
        occSubTable["timeSlotId"] = timeSlotTable[timeSlotTable["time"].isin(time_index)].index
        occupancyTable = pd.concat([occupancyTable, occSubTable])


display(occupancyTable)

df.to_csv("data.csv", index=False)

,time,sessionId,courseId,weekdayId,studentCount,timeSlotId
0,11:00:00,2252,000014,2,51,180
1,11:01:00,2252,000014,2,51,181
2,11:02:00,2252,000014,2,51,182
3,11:03:00,2252,000014,2,51,183
4,11:04:00,2252,000014,2,51,184
...,...,...,...,...,...,...
71,13:41:00,2252,000223,4,75,341
72,13:42:00,2252,000223,4,75,342
73,13:43:00,2252,000223,4,75,343
74,13:44:00,2252,000223,4,75,344
